<a href="https://colab.research.google.com/github/Machaniharika/21311A12L5/blob/main/gesture%20recognizatiom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install opencv-python-headless tensorflow matplotlib


In [6]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from IPython.display import Image
import cv2
import numpy as np
import PIL.Image
import io
import base64


def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = '📸 Capture';
            div.appendChild(capture);
            document.body.appendChild(div);

            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            const video = document.createElement('video');
            video.style.display = 'block';
            document.body.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Resize the output to fit the video element.
            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(track => track.stop());
            div.remove();
            video.remove();

            const dataUrl = canvas.toDataURL('image/jpeg', quality);
            return dataUrl;
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = io.BytesIO(base64.b64decode(data.split(',')[1]))
    img = PIL.Image.open(binary)
    img.save(filename)
    return filename


In [7]:
filename = take_photo('gesture_sample.jpg')
print('Saved to:', filename)

<IPython.core.display.Javascript object>

Saved to: gesture_sample.jpg


In [15]:
from google.colab import files
uploaded = files.upload()  # This will open a file dialog to upload files


Saving gesture_data.zip to gesture_data.zip


In [16]:
!unzip gesture_data.zip -d /content/


Archive:  gesture_data.zip
   creating: /content/gesture_data/
   creating: /content/gesture_data/dislike/
  inflating: /content/gesture_data/dislike/dislike.jpg  
   creating: /content/gesture_data/peace/
  inflating: /content/gesture_data/peace/peace.jpg  
   creating: /content/gesture_data/thumb/
  inflating: /content/gesture_data/thumb/thumb.jpg  


In [17]:
categories = os.listdir('/content/gesture_data')

In [18]:
import os
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical

def load_images_from_folder(folder, label, size=(64, 64)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, size)
            images.append(img)
            labels.append(label)
    return images, labels

# Example: Upload folders named "peace", "thumbs_up" into /content/gesture_data/
# You can zip them and upload using the left panel in Colab.
gesture_data_path = '/content/gesture_data'
categories = os.listdir(gesture_data_path)

X, y = [], []
for idx, category in enumerate(categories):
    imgs, labels = load_images_from_folder(os.path.join(gesture_data_path, category), idx)
    X.extend(imgs)
    y.extend(labels)

X = np.array(X) / 255.0
X = np.expand_dims(X, axis=-1)
y = to_categorical(y)


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(64,64,1)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(categories), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=10, validation_split=0.2)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 1.1510 - val_accuracy: 0.0000e+00 - val_loss: 1.9076
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step - accuracy: 0.5000 - loss: 0.7454 - val_accuracy: 0.0000e+00 - val_loss: 2.9085
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step - accuracy: 0.5000 - loss: 0.6434 - val_accuracy: 0.0000e+00 - val_loss: 4.0366
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - accuracy: 0.5000 - loss: 0.6005 - val_accuracy: 0.0000e+00 - val_loss: 5.0838
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.5000 - loss: 0.5544 - val_accuracy: 0.0000e+00 - val_loss: 6.0600
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 1.0000 - loss: 0.4897 - val_accuracy: 0.0000e+00 - val_loss: 7.0926
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - accuracy: 1.0000 - loss: 0.4185 - val_accuracy: 0.0000e+00 - val_loss: 8.1943
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - accuracy: 1.0000 - loss: 0.3738 - val

In [20]:
def predict_single_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (64,64)) / 255.0
    img = np.expand_dims(img, axis=(0, -1))
    pred = model.predict(img)
    return categories[np.argmax(pred)]

# Example usage:
print("Prediction:", predict_single_image("gesture_sample.jpg"))


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Prediction: peace
